In [5]:
import tkinter as tk
from tkinter import messagebox

MAGIC_SUM = 15
GRID_SIZE = 3

magic_squares = []
current_index = 0

def is_valid(square):
    for i in range(GRID_SIZE):
        if sum(square[i]) != MAGIC_SUM:
            return False
        if sum(square[j][i] for j in range(GRID_SIZE)) != MAGIC_SUM:
            return False
    if sum(square[i][i] for i in range(GRID_SIZE)) != MAGIC_SUM:
        return False
    if sum(square[i][GRID_SIZE - 1 - i] for i in range(GRID_SIZE)) != MAGIC_SUM:
        return False
    return True

from itertools import permutations

def generate_all_magic_squares():
    global magic_squares
    magic_squares = []
    for perm in permutations(range(1, 10)):
        square = [list(perm[i:i+3]) for i in range(0, 9, 3)]
        if is_valid(square) and square not in magic_squares:
            magic_squares.append(square)

def show_square(index):
    square = magic_squares[index]
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            entry_grid[i][j].config(state='normal')
            entry_grid[i][j].delete(0, tk.END)
            entry_grid[i][j].insert(0, str(square[i][j]))
            entry_grid[i][j].config(state='readonly')
    status_label.config(text=f"Solution {index + 1} of {len(magic_squares)}")

def on_next():
    global current_index
    if not magic_squares:
        return
    current_index = (current_index + 1) % len(magic_squares)
    show_square(current_index)

def on_solve():
    global current_index
    generate_all_magic_squares()
    if magic_squares:
        current_index = 0
        show_square(current_index)
    else:
        status_label.config(text="No magic squares found.")

def on_clear():
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            entry_grid[i][j].config(state='normal')
            entry_grid[i][j].delete(0, tk.END)
    status_label.config(text="")

def on_check():
    try:
        square = []
        used_numbers = set()
        for i in range(GRID_SIZE):
            row = []
            for j in range(GRID_SIZE):
                val_str = entry_grid[i][j].get().strip()
                val = int(val_str)
                if not (1 <= val <= 9):
                    raise ValueError("Numbers must be between 1 and 9.")
                if val in used_numbers:
                    raise ValueError("Numbers must not repeat.")
                used_numbers.add(val)
                row.append(val)
            square.append(row)
    except ValueError as e:
        messagebox.showerror("Invalid Input", f"Invalid input: {e}")
        return

    if is_valid(square):
        messagebox.showinfo("Success!", " Congratulations! This is a valid magic square.")
    else:
        messagebox.showinfo("Try Again", " This is NOT a valid magic square. Keep trying!")

# Tooltip helper for buttons
class CreateToolTip(object):
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tipwindow = None
        widget.bind("<Enter>", self.show_tip)
        widget.bind("<Leave>", self.hide_tip)

    def show_tip(self, event=None):
        if self.tipwindow or not self.text:
            return
        x = y = 0
        x = self.widget.winfo_rootx() + 25
        y = self.widget.winfo_rooty() + 20
        self.tipwindow = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)
        tw.wm_geometry(f"+{x}+{y}")
        label = tk.Label(
            tw, text=self.text, justify=tk.LEFT,
            background="#fff9db", relief=tk.SOLID, borderwidth=1,
            font=("tahoma", "8", "normal"), fg="#6b4c3b"
        )
        label.pack(ipadx=5, ipady=2)

    def hide_tip(self, event=None):
        if self.tipwindow:
            self.tipwindow.destroy()
        self.tipwindow = None

# Unique color palette
BG_COLOR = "#1f2937"          # Dark slate
GRID_BG = "#2a3948"           # Slightly lighter dark slate
ENTRY_BG = "#f4f1de"          # Warm off-white
ENTRY_FG = "#3d405b"          # Deep blue-gray
BUTTON_BG = "#e07a5f"         # Coral
BUTTON_FG = "#f4f1de"         # Off-white text on buttons
BUTTON_ACTIVE_BG = "#d65a3a"  # Darker coral for active
CLEAR_BUTTON_BG = "#f2cc8f"   # Mustard yellow
CLEAR_BUTTON_ACTIVE = "#d4b86a"
STATUS_FG = "#ffe8d6"         # Pale warm for status text

root = tk.Tk()
root.title("3x3 Magic Square - Manual Solver")
root.resizable(False, False)
root.configure(bg=BG_COLOR)

grid_frame = tk.Frame(root, bg=GRID_BG, bd=3, relief="ridge", padx=12, pady=12)
grid_frame.grid(row=0, column=0, columnspan=3, padx=25, pady=20)

entry_grid = [[None] * GRID_SIZE for _ in range(GRID_SIZE)]

for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        entry = tk.Entry(grid_frame, width=4, font=('Helvetica', 24, 'bold'),
                         justify='center', bd=3, relief='groove',
                         bg=ENTRY_BG, fg=ENTRY_FG, insertbackground=ENTRY_FG)
        entry.grid(row=i, column=j, padx=8, pady=8)
        entry_grid[i][j] = entry

btn_font = ('Helvetica', 13, 'bold')

check_button = tk.Button(root, text="Check", command=on_check,
                         font=btn_font, bg=BUTTON_BG, fg=BUTTON_FG,
                         activebackground=BUTTON_ACTIVE_BG, relief='raised', bd=3, width=12)
check_button.grid(row=1, column=0, padx=10, pady=12)

solve_button = tk.Button(root, text="Solve All", command=on_solve,
                         font=btn_font, bg=BUTTON_BG, fg=BUTTON_FG,
                         activebackground=BUTTON_ACTIVE_BG, relief='raised', bd=3, width=12)
solve_button.grid(row=1, column=1, padx=10, pady=12)

next_button = tk.Button(root, text="Next Solution", command=on_next,
                        font=btn_font, bg=BUTTON_BG, fg=BUTTON_FG,
                        activebackground=BUTTON_ACTIVE_BG, relief='raised', bd=3, width=12)
next_button.grid(row=1, column=2, padx=10, pady=12)

clear_button = tk.Button(root, text="Clear", command=on_clear,
                         font=btn_font, bg=CLEAR_BUTTON_BG, fg=ENTRY_FG,
                         activebackground=CLEAR_BUTTON_ACTIVE, relief='raised', bd=3, width=38)
clear_button.grid(row=2, column=0, columnspan=3, pady=(0, 20))

status_label = tk.Label(root, text="", font=('Helvetica', 11, 'italic'),
                        bg=BG_COLOR, fg=STATUS_FG)
status_label.grid(row=3, column=0, columnspan=3)

# Tooltips
CreateToolTip(check_button, "Check if your input is a valid magic square")
CreateToolTip(solve_button, "Show the first magic square solution")
CreateToolTip(next_button, "Cycle through all magic square solutions")
CreateToolTip(clear_button, "Clear the grid for a new try")

root.mainloop()
